In [25]:
from collections import defaultdict

In [80]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=6)

<IB connected to 127.0.0.1:7497 clientId=6>

In [115]:
df_ratio

,adr_num_per_stock,stock_num_per_adr
Pair,,
ATHE_ATH,0.016667,60.0
GENE_GTG,0.001667,600.0
IMMP_IMM,0.100000,10.0
IMRN_IMC,0.025000,40.0
JHX_JHX,1.000000,1.0
KZIA_KZA,0.100000,10.0
MESO_MSB,0.200000,5.0
PLL_PLL,0.010000,100.0
WBK_WBC,1.000000,1.0


Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has bee

In [114]:
##### Initialization of global variables here ###

order_books = defaultdict(set)
df_ratio = pd.read_csv("ratio_dict.csv", index_col = 0).set_index("Pair")
df_global = pd.DataFrame({"date":[]})


country_info = {
    "Australia":{"Currency": "AUD", 
                "Exchange": "SMART",
                "Pairs": aus_adr_underlying_pairs},
    
    "Japan":{"Currency": "JPY", 
             "Exchange": "TSEJ",
             "Pairs": jpn_adr_underlying_pairs},
    
    "China":{"Currency": "HKD", 
         "Exchange": "SEHK",
         "Pairs": hk_adr_underlying_pairs},
    
}





In [93]:



def place_order(country, security, sectype, side, quantity, price):
    global order_books
    exchange = country_info[country]["Exchange"]
    currenct = country_info[country]["Currency"]
    
    # Define order 
    contract = Contract(symbol = security, secType = 'STK', exchange =exchange, currency = currency)
    order = LimitOrder(side,quantity,price)
    
    # Place the order
    msg = ib.placeOrder(contract, order)
    orderId = msg.order.orderId
    
    # Add the orderId + quantity to orderbook
    name =  security +"_"+ sectype
    order_books[name].add(orderId)
    
    

def check_open_order(security, sectype = "underlying"):
    """
    Check if we have open order for this particular security
    Args: 
    security (str) name of security
    sectype(str): takes th evalue either "adr" or "underlying"
    
    return: the quantity of open orders (orders that we wanted to trade but not traded) of a particular security
    
    """
    global order_books
    ct = 0
    lookup = security +"_"+ sectype
    all_orders = order_books[lookup]
    open_orders = ib.openOrders()
    for o in openOders:
        if o.orderId in all_orders:
            ct += o.totalQuantity
        
    return ct
        
    

Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained. All data farms are connected: hfarm; usfarm.nj; eufarm; jfarm; usfuture; cashfarm; usfarm; euhmds; cashhmds; fundfarm; ushmds; secdefil.
Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained. All data farms are connected: hfarm; usfarm.nj; eufarm; jfarm; usfuture; cashfarm; usfarm; euhmds; cashhmds; fundfarm; ushmds; secdefil.


In [113]:

def convert_price_asian_close():
    global df_global, df_forex, df_ratio, country_info
    
    # We use the last 
    for key in country_info:
        pairs = country_info[key]
        for pair in pairs:
            underlying_col_name = f"underlying_{pair[1]}_close"
            pair_name =  pair[0] + "_" + pair[1]
            ratio = df_ratio.loc[pair_name,"adr_num_per_stock"]
            
            #
            # I will update this part after knowing what the forex data look like
            #
            
            price_in_usd = 0
            
            df_global.at[len(df_global)-1, underlying_col_name] = price_in_usd
            
            
    
    
    
    

In [33]:
# from ib_insync import *
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import math
import datetime 
import statsmodels.formula.api as smf 
from datetime import date, time, datetime, timedelta
import seaborn as sns
import random
# from ta import add_all_ta_features
# from ta.utils import dropna
# from ta.trend import *

In [106]:
#define a function to get IB data. endDate is the last date of the historical data 

def get_data(contract, history, freq, side, endDate =''):
    bar = ib.reqHistoricalData(
        contract,
        endDateTime = endDate,
        durationStr=history,
        barSizeSetting= freq,
        whatToShow=side,
        useRTH=True,
        formatDate=1)
    return util.df(bar)

history = '300 S'
freq = '1 min'
side ='Trades'

# write getter functions?

In [47]:
# Initialization of hard coded data

aus_adr_underlying_pairs = [
    ["ATHE", "ATH"],["GENE", "GTG"],["IMMP", "IMM"],["IMRN","IMC"],["JHX", "JHX"],["KZIA", "KZA"],
    ["MESO","MSB"],["PLL", "PLL"],["WBK","WBC"]
]

hk_adr_underlying_pairs = [
    ["ACH", "2600"],["BGNE", "6160"],["CEA","670"],["HNP", "902"],["LFC", "2628"],["PTR", "857"],
    ["SHI", "338"],["SNP","386"],["ZNH", "1055"]
]

jpn_adr_underlying_pairs = [
    ["CAJ", "7751"],["HMC","7267"],["IX","8591"],["MFG","8411"],["MUFG","8306"],["NMR","8604"],
    ["SMFG","8316"],["SONY","6758"],["TAK", "4502"],["TM", "7203"]
]

country_list = [
    (aus_adr_underlying_pairs, "AUD", "SMART", "Australia" ),
    (hk_adr_underlying_pairs, "HKD", "SEHK", "China"),
    (jpn_adr_underlying_pairs, "JPY", "TSEJ", "Japan")
]


# functions return the market opening time adjusted for DST

us_dst = {
        2021: (datetime(2021, 3, 14), datetime(2021, 11, 7)),
        2020: (datetime(2020, 3, 8),  datetime(2020, 11, 1)),
        2019: (datetime(2019, 3, 10), datetime(2019, 11, 3)),
        2018: (datetime(2018, 3, 11), datetime(2018, 11, 4)),
        2017: (datetime(2017, 3, 12), datetime(2017, 11, 5)),
        2016: (datetime(2016, 3, 13), datetime(2016, 11, 6)),
        2015: (datetime(2015, 3, 9),  datetime(2015, 11, 2)),
    } 

def next_market_open_day(date_now):
    '''
    Returns today if Mon - Thurs, returns next Mon if Fri thru Sun
    '''
    while date_now.weekday()>4:
        date_now += timedelta(days = 1)
    return date_now

def AUS_opening(date_now):
    date_now = next_market_open_day(date_now)
    us_start, us_end = us_dst[date_now.year]
    if us_start < date_now and date_now < us_end:
        return datetime.combine(date_now,time(10,0)) - timedelta(hours = 14)
    else:
        return datetime.combine(date_now, time(10,0)) - timedelta(hours = 15)

def AUS_closing(date_now):
    open_time = AUS_opening(date_now)
    return open_time + timedelta(hours = 6)

def HK_opening(date_now):
    date_now = next_market_open_day(date_now)
    start, end = us_dst[date_now.year]
    if start < date_now and date_now < end:
        return datetime.combine(date_now,time(9,30)) - timedelta(hours = 12)
    else:
        return datetime.combine(date_now, time(9,30)) - timedelta(hours = 13)
    
def HK_closing(date_now):
    open_time = HK_opening(date_now)
    return open_time + timedelta(hours = 6.5)
    
def JPN_opening(date_now):
    date_now = next_market_open_day(date_now)
    start, end = us_dst[date_now.year]
    if start < date_now and date_now < end:
        return datetime.combine(date_now,time(9,0)) - timedelta(hours = 13)
    else:
        return datetime.combine(date_now,time(9,0)) - timedelta(hours = 14)
    
def JPN_closing(date_now):
    open_time = JPN_opening(date_now)
    return open_time + timedelta(hours = 6)

def US_opening(date_now):
    return datetime.combine(next_market_open_day(date_now),time(9,30))

def US_closing(date_now):
    return datetime.combine(next_market_open_day(date_now),time(16,0))

In [6]:
HK_opening(datetime.now())

datetime.datetime(2021, 4, 19, 21, 30)

In [9]:
contract = Contract(symbol = "AAPL", secType = 'STK', exchange ="SMART", currency = "USD")
df = get_data(contract, history, freq, side, endDate ='')

Error 162, reqId 4: Historical Market Data Service error message:No market data permissions for ASX STK, contract: Contract(secType='STK', symbol='ATH', exchange='SMART', currency='AUD')


In [53]:
ratio_dict = defaultdict(dict)
for key in country_info:
    pairs = country_info[key]["Pairs"]
    for pair in pairs:
        
        name = pair[0]+ "_" + pair[1]
        data_processing(key, name, fx_dict, forex_bps = 10, adjust_forex_expense = True)
        ratio_dict[name]["adr_num_per_stock"] = data_processing(key, name, fx_dict, forex_bps = 10, adjust_forex_expense = True).loc[1,"adr_num_per_unit"]
        ratio_dict[name]["stock_num_per_adr"] = data_processing(key, name, fx_dict, forex_bps = 10, adjust_forex_expense = True).loc[1,"stock_num_per_unit"]
        
    

In [64]:
ratio_dict_rewrite = {"Pair":[], "adr_num_per_stock":[], "stock_num_per_adr":[] }

for key in ratio_dict:
    ratio_dict_rewrite["Pair"].append(key)
    ratio_dict_rewrite["adr_num_per_stock"].append(ratio_dict[key]["adr_num_per_stock"])
    ratio_dict_rewrite["stock_num_per_adr"].append(ratio_dict[key]["stock_num_per_adr"])
    


In [74]:
ratio_df = pd.DataFrame(ratio_dict_rewrite)
ratio_df.to_csv("ratio_dict.csv")

In [55]:
for key in ratio_dict:
    r1, r2 = ratio_dict[key]["adr_num_per_stock"],ratio_dict[key]["stock_num_per_adr"]
    if r1 == r2 == 1:
        pass
    elif r1 > 1:
        ratio_dict[key]["stock_num_per_adr"] = 1/r1
    else:
        ratio_dict[key]["adr_num_per_stock"] = 1/r2

In [126]:

# Store all the historical adr/underlying open and close data


def pull_asian_close():
    global df_global, country_info

    today = datetime.now().date()
    history = "1 D"
    freq = "1 Day"
    side = 'Trades'
    
    df_asian_close = pd.DataFrame({"date":[]})
    for key in country_info.keys():
        pairs = country_info[key]["Pairs"]
        exchange = country_info[key]["Exchange"]
        currency = country_info[key]["Currency"]
        
        for p in pairs:
            underlying = p[1]
            contract = Contract(symbol = underlying, secType = 'STK', exchange = exchange, currency = currency)
            try:
                df = get_data(contract, history, freq, side, endDate =today)
                df = df.loc[:, ["date", "close"]]
                col_name = "underlying_" + underlying + "_close"
                df = df.rename(columns = {"close":col_name})
                print(df)
                df_asian_close = pd.merge(df, df_asian_close, how = "outer")
            except:
                pass
            
    df_global = pd.concat([df_global, df_asian_close])

    
    
def pull_us_close():
    global df_global, country_info

    today = datetime.now().date()
    history = "1 D"
    freq = "1 Day"
    side = 'Trades'
    
    df_us_close = pd.DataFrame({"date":[]})
    for key in country_info.keys():
        pairs = country_info[key]["Pairs"]
        exchange = "SMART"
        currency = "USD"
        
        for p in pairs:
            print(p)
            underlying = p[0] # get the ticker of the ADR
            contract = Contract(symbol = underlying, secType = 'STK', exchange = exchange, currency = currency)
            df = get_data(contract, history, freq, side, endDate ='')
            df = df.loc[:, ["date", "close"]]
            col_name = "adr_" + underlying + "_close"
            df = df.rename(columns = {"close":col_name})
            df_us_close = pd.merge(df, df_us_close, how = 'outer')
    
    df_global = pd.merge(df_global, df_us_close,how = 'outer')



In [131]:
forex_pairs = [("AUD", "USD"),  ("USD", "HKD"), 
               ("USD","JPY")]

In [132]:
pull_forex_data()

ValueError: not enough values to unpack (expected 3, got 2)

Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained. All data farms are connected: hfarm; usfarm.nj; eufarm; jfarm; usfuture; cashfarm; usfarm; euhmds; hkhmds; cashhmds; fundfarm; ushmds; secdefil.
Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained. All data farms are connected: hfarm; usfarm.nj; eufarm; jfarm; usfuture; cashfarm; usfarm; euhmds; hkhmds; cashhmds; fundfarm; ushmds; secdefil.


In [129]:
def pull_forex_data(history = "1 D", freq = "1 min", side = "BID_ASK", end_date = "" ):
    '''
    Updates all three forex pairs
    '''
    
    global df_global, country_info, forex_pairs, test

    
    df_forex = pd.DataFrame({"date":[]})
    for currency1, currency2, asian_opening in forex_pairs:
        contract = Contract(symbol = currency1, secType = 'CASH', exchange = "IDEALPRO", currency = currency2)
        df = get_data(contract, history, freq, side)

        if currency2 == "USD":
            df["mid"] = (df["open"] + df["close"])/2
            forex_string = f"{currency1}.{currency2}"
        else:
            df["mid"] = 2/(df["open"] + df["close"])
            forex_string = f"{currency2}.{currency1}"

        df = df.loc[:, ["date", "MID"]]
        df_us_open = df[[(minute == US_opening(minute)) for minute in df["date"]]]

        df_asian_open = df[[(minute == asian_opening(minute)) for minute in df["date"]]]


        df_us_open = df_us_open.rename(columns = {"close":f"{forex_string}_MID_US_open"})
        df_asian_open = df_asian_open.rename(columns = {"close":f"{forex_string}_MID_{currency2}_open"})

        df_forex = pd.merge(df_forex, df_us_open, how = "outer")
        df_forex = pd.merge(df_forex, df_asian_open, how = "outer")

        df_global = pd.concat([df_global, df_forex])
        df_global.to_csv("global_df.csv")


In [ ]:
# 8.30 am
    # Pull Historical Forex Data (Minute, for lookback window)
    # Pull Close Data for Asian Market at 9am (Before US Market opens)
# 9.28 am 
    # Pull Forex Data for three currencies (For Z score)
    # Calculate Z Score 
    # for each pair:
        # if invested:
            # if risk condition or exit condition:
                # liquidate
            # else:
                # continue
        # if entry condition:
            # determine order size from risk metric and ADT over last 5 days
            # if 0: no order made
            # else: 
                # Define contract
                # make order (What order? Market Order?)


# 6 pm
    # Pull Close Data for US Market at 6pm (Before Asian Market opens)
    # Pull Historical Forex Data

# 2 min before Asian Market Open
    # Pull Forex Data
    # Calculate Z score using market close and minute forex data

In [ ]:
# initialize data lists, hyperparameters etc

In [ ]:
def get_

In [ ]:
#update date
while True:
    # keep track of what day it is
    today = datetime.date() # ideally the algorithm starts at 8am in the morning
    
    # keep track of how many times we pull data for calibration
    bef_us_pull = False
    bef_us_zscore = False
    bef_asia_pull = False
    bef_jpn_zscore = False
    bef_aus_zscore = False
    bef_hk_zscore = False
    
    # Various Open and Close hours for various markets
    
    US_open = US_opening(today)
    US_close = US_closing(today)
    HK_open = HK_opening(today + 1)
    HK_close = HK_closing(today + 1)
    JPN_open = JPN_opening(today + 1)
    JPN_close = JPN_closing(today + 1)
    AUS_open = AUS_opening(today + 1)
    AUS_close = AUS_closing(today + 1)
    
    while now< US_close and now < HK_close and now < JPN_close and now < AUS_close:
        
        # Before US Market Open pull historical data
        if not bef_us_pull and now > US_open - timedelta(hours = 1):
            bef_us_pull = True
            # Pull minute forex data
            # Pull asian close for underlying stocks
        
        # Right before US Market Open for Z Score
        if not bef_us_zscore and now > US_open - timedelta(minutes = 4):
            bef_us_zscore = True
            # Calculate Z score and make decision on trade and size
            
        if US_open < now and now < US_close:
            # Pull minute data to check whether to liquidate positions
            # Check pending limit orders
            
        
        # before asian markets open, pull us market close
        if not bef_asia_pull and now > US_close + timedelta(hours = 1):
            bef_asia_pull = True
            # pull us close
        
        # before Hong Kong market open, pull forex to calculate Z score
        if not bef_hk_zscore and now > HK_open - timedelta(minutes = 4):
            bef_hk_zscore = True
            # calculate Z score and make decision on trade and size
        
        # check on Hong Kong Stocks
        if HK_open < now and now < HK_close:
            # Pull minute data to check whether to liquidate positions
            # Check pending limit orders
        
        # before Japan market open, pull forex to calculate Z score
        if not bef_jpn_zscore and now > JPN_opening(today) - timedelta(minutes = 4):
            bef_JPN_zscore = True
            # calculate Z score and make decision on trade and size
        
        # check on Japan stocks
        if JPN_open < now and now < JPN_close:
            # Pull minute data to check whether to liquidate positions
            # Check pending limit orders
            
        # before Australia market open, pull forex to calculate Z score            
        if not bef_aus_zscore and now > AUS_opening(today) - timedelta(minutes = 4):
            bef_aus_zscore = True
        
        # check on Australian Stocks
        if AUS_open < now and now < AUS_close:
            # Pull minute data to check whether to liquidate positions
            # Check pending limit orders         
        

        ib.sleep(120) # Loop every two minutes
        now = datetime.now()

In [29]:
adr_symbol = "CAJ"
adr_exchange = 'SMART'
adr_currency = "USD"

underlying_symbol = '7751'
underlying_exchange = 'TSEJ'
underlying_currency = 'JPY'
underlying_country = "Japan"

adr_contract = Contract(symbol = adr_symbol, secType = 'STK', exchange = adr_exchange, currency = adr_currency)
ib.qualifyContracts(adr_contract)
underlying_contract = Contract(symbol = underlying_symbol, secType = 'STK', exchange = underlying_exchange, currency = underlying_currency)
ib.qualifyContracts(underlying_contract)
order = MarketOrder("BUY", 100)
ib.placeOrder(underlying_contract, order)
# adr_df = get_data(adr_contract, history, freq, side)
# print(adr_df.head(3))
# adr_df.to_csv(folder_name + "adr.csv")

Trade(contract=Contract(secType='STK', conId=13905840, symbol='7751', exchange='TSEJ', primaryExchange='TSEJ', currency='JPY', localSymbol='7751.T', tradingClass='7751'), order=MarketOrder(orderId=22, clientId=15, action='BUY', totalQuantity=100), orderStatus=OrderStatus(orderId=22, status='PendingSubmit', filled=0, remaining=0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2021, 4, 20, 1, 2, 14, 489696, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='')])